# Quick Start
To get started with `swmmio`, you'll need a EPA SWMM input file (.inp). We'll an example model provided within `swmmio` and see what it looks like to get a `links` DataFrame.  

In [1]:
from swmmio.tests.data import MODEL_FULL_FEATURES_PATH
import swmmio

# instantiate a model object
model = swmmio.Model(MODEL_FULL_FEATURES_PATH)

# get the data related to subcatchments
model.links.dataframe

,InletNode,OutletNode,...,Barrels,coords
Name,,,,,
C1:C2,J1,J2,...,1.0,"[(0.0, 0.0), (238.75, -53.332)]"
C3,J3,J4,...,NaN,"[(459.058, -113.145), (671.391, -163.985)]"
C2,J2,J3,...,NaN,"[(238.75, -53.332), (459.058, -113.145)]"


Now let's use the lower-level `swmmio.Model.inp` API to access and modify the sections of the model. We'll change the outfall type to FIXED and set a stage elevation. 


In [2]:
# here is the exsiting outfalls DataFrame
model.inp.outfalls

,InvertElev,OutfallType,StageOrTimeseries
Name,,,
J4,0,FREE,NO


In [3]:
# modify the outfall DataFrame
model.inp.outfalls.loc['J4', 'OutfallType'] = 'FIXED'
model.inp.outfalls.loc['J4', 'StageOrTimeseries'] = 3.0

model.inp.outfalls

,InvertElev,OutfallType,StageOrTimeseries
Name,,,
J4,0,FIXED,3.0


We can then save our updated model to a new .inp file, then instantiate a new `swmmio.Model` object. 

In [4]:
model.inp.save('new_model.inp')

# instantiate a new Model object with the modified inp file
new_model = swmmio.Model('new_model.inp')

# see the changes in the higher-level nodes DataFrame
new_model.nodes.dataframe 

,InvertElev,MaxDepth,...,EvapFrac,coords
Name,,,,,
J1,20.728,15.0,...,NaN,"[(0.0, 0.0)]"
J3,6.547,15.0,...,NaN,"[(459.058, -113.145)]"
J4,0.000,NaN,...,NaN,"[(671.391, -163.985)]"
J2,13.392,NaN,...,0.0,"[(238.75, -53.332)]"


## Building Variations of Models
Starting with a base SWMM model, other models can be created by inserting altered data into a new inp file. Useful for sensitivity analysis or varying boundary conditions, models can be created using a fairly simple loop.

For example, climate change impacts can be investigated by creating a set of models with varying outfall Fixed Stage elevations:


In [5]:
import os

# path to a SWMM model from swmm-nrtestsuite
model_path = 'https://raw.githubusercontent.com/USEPA/swmm-nrtestsuite/refs/heads/dev/public/examples/Example3.inp'

# initialize a baseline model object
baseline = swmmio.Model(model_path)

rise = 0.0 # set the starting sea level rise condition

# set the outfall type and initial outfall stage elevation
baseline.inp.outfalls.loc[:, 'OutfallType'] = 'FIXED'
baseline.inp.outfalls.loc[:, 'StageOrTimeseries'] = 576

# create models up to 5ft of sea level rise.
while rise <= 5:

    # create a dataframe of the model's outfalls
    outfalls = baseline.inp.outfalls

    # add the current rise to the outfalls' stage elevation
    outfalls.loc[:, 'StageOrTimeseries'] = outfalls.loc[:, 'StageOrTimeseries'] + rise
    baseline.inp.outfalls = outfalls
    
    # create the filename for the new model
    newfilepath = os.path.join(baseline.inp.dir, f'{baseline.inp.name}_{rise}_SLR.inp')
    
    # Overwrite the OUTFALLS section of the new model with the adjusted data
    baseline.inp.save(newfilepath)

    # increase sea level rise for the next loop
    rise += 0.25

# check the outfalls of the last SLR scneario
model = swmmio.Model(newfilepath)
model.inp.outfalls

,InvertElev,OutfallType,StageOrTimeseries
Name,,,
KRO2005,574.32,FIXED,628.5
PSO,548.36,FIXED,628.5
